<a href="https://colab.research.google.com/github/swastikbanerjee/AML_Assg/blob/main/RidgeLassoRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [39]:
data = pd.read_csv("/content/Program_2_Dataset.csv")

In [40]:
data.shape

(506, 14)

In [41]:
data.head()

,Crime_Rate,Zone_proportion,Non_Retail_Acres,CHAS,NOX,Avg_rooms,AGE,Distance,RAD,Property_TaxRate,PTRATIO,B,LSTAT,MValue
0,0.06860,0.0,2.89,0,0.445,7.416,62.5,3.4952,2,276,18.0,396.90,6.19,33.2
1,0.14866,0.0,8.56,0,0.520,6.727,79.9,2.7778,5,384,20.9,394.76,9.42,27.5
2,0.11432,0.0,8.56,0,0.520,6.781,71.3,2.8561,5,384,20.9,395.58,7.67,26.5
3,0.22876,0.0,8.56,0,0.520,6.405,85.4,2.7147,5,384,20.9,70.80,10.63,18.6
4,0.21161,0.0,8.56,0,0.520,6.137,87.4,2.7147,5,384,20.9,394.47,13.44,19.3


In [42]:
data.isnull().sum()

Crime_Rate          0
Zone_proportion     0
Non_Retail_Acres    0
CHAS                0
NOX                 0
Avg_rooms           0
AGE                 0
Distance            0
RAD                 0
Property_TaxRate    0
PTRATIO             0
B                   0
LSTAT               0
MValue              0
dtype: int64

In [43]:
x = data.drop(['MValue'],axis=1)
y = data['MValue']

In [44]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [45]:
from sklearn.linear_model import LinearRegression
modelLR = LinearRegression()
modelLR.fit(x_train,y_train)

LinearRegression()

In [46]:
print("Intercept: ",modelLR.intercept_,"\nCoefficients: ",modelLR.coef_)

Intercept:  37.09747556859814 
Coefficients:  [-1.12766013e-01  3.74631992e-02  3.22219699e-02  1.93444095e+00
 -1.76337882e+01  3.92197587e+00 -2.49147983e-03 -1.57540275e+00
  3.06232262e-01 -1.31759199e-02 -9.89726184e-01  1.03390007e-02
 -5.31237978e-01]


In [47]:
modelLR.score(x_test,y_test)

0.7395724246395499

In [48]:
from sklearn.linear_model import Ridge
modelR = Ridge(alpha=1)
modelR.fit(x,y)

Ridge(alpha=1)

In [49]:
print("Intercept: ",modelR.intercept_,"\nCoefficients: ",modelR.coef_)

Intercept:  31.5976698182738 
Coefficients:  [-1.04595278e-01  4.74432243e-02 -8.80467889e-03  2.55239322e+00
 -1.07770146e+01  3.85400020e+00 -5.41453810e-03 -1.37265353e+00
  2.90141589e-01 -1.29116463e-02 -8.76074394e-01  9.67327945e-03
 -5.33343225e-01]


In [50]:
modelR.score(x_test,y_test)

0.7522607268336814

In [60]:
mse = mean_squared_error(y, modelR.predict(x))
print("Mean Squared Error on the Entire Dataset:", mse)

Mean Squared Error on the Entire Dataset: 22.044452243088198


In [51]:
from sklearn.model_selection import cross_val_score, KFold
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
ridge_scores = cross_val_score(modelR, x, y, cv=kf)
print(f'Ridge Regression Cross-Validation Scores: {ridge_scores}')
print(f'Mean Cross-Validation Score: {np.mean(ridge_scores)}')

Ridge Regression Cross-Validation Scores: [0.73756245 0.67886458 0.803029   0.73383259 0.62846038]
Mean Cross-Validation Score: 0.7163498013765897


Comparison of CV score with R-squared on Full Dataset:

If the mean cross-validation score is significantly lower than the R-squared score obtained on the full dataset, it could indicate overfitting. Overfitting occurs when a model learns the noise in the training data rather than the underlying patterns.
If the mean cross-validation score is close to the R-squared score on the full dataset, it suggests that the model is likely to generalize well to new, unseen data

In [61]:
from sklearn.model_selection import GridSearchCV
param_grid = {'alpha': [0.1, 1.0, 10.0]}
grid_search = GridSearchCV(modelR, param_grid, cv=10,  scoring='neg_mean_squared_error') #default cv is 5 if not mentioned
grid_search.fit(x, y)
print("Best hyperparameters:", grid_search.best_params_)
print("Best mean cross-validated score:", grid_search.best_score_)

Best hyperparameters: {'alpha': 10.0}
Best mean cross-validated score: -30.663590235424756



The scoring parameter in GridSearchCV is used to specify the metric that should be optimized to select the best hyperparameters for your model. This parameter determines the evaluation criterion for each combination of hyperparameters during the grid search.

The choice of the scoring metric depends on the nature of your machine learning problem. For example, in a regression problem, you might use metrics like Mean Squared Error ('neg_mean_squared_error') or R-squared ('r2'). In a classification problem, you might use metrics like accuracy ('accuracy'), precision ('precision'), recall ('recall'), F1-score ('f1'), or area under the receiver operating characteristic curve ('roc_auc').

The neg_mean_squared_error is a scoring metric used in scikit-learn for regression problems. It represents the negative of the mean squared error (MSE) of the predictions. The negative sign is used because scikit-learn's grid search and cross-validation modules are designed to maximize the scoring metric, but in the case of MSE, lower values are better.

The negative of MSE is used in neg_mean_squared_error to align with the convention of maximizing the scoring metric. So, when you see a result with neg_mean_squared_error, a more negative value actually corresponds to a better model.

In [62]:
from sklearn.metrics import mean_squared_error
modelRCV = Ridge(alpha=10)
modelRCV.fit(x,y)
print("Intercept: ",modelRCV.intercept_,"\nCoefficients: ",modelRCV.coef_)
mse = mean_squared_error(y, modelRCV.predict(x))
print("Mean Squared Error on the Entire Dataset:", mse)

Intercept:  27.46788496414103 
Coefficients:  [-0.10143535  0.0495791  -0.0429624   1.95202082 -2.37161896  3.70227207
 -0.01070735 -1.24880821  0.2795956  -0.01399313 -0.79794498  0.01003684
 -0.55936642]
Mean Squared Error on the Entire Dataset: 22.66036355563931


In [52]:
from sklearn.linear_model import Lasso
modelL = Lasso(alpha=1)
modelL.fit(x,y)

Lasso(alpha=1)

In [53]:
print("Intercept: ",modelL.intercept_,"\nCoefficients: ",modelL.coef_)

Intercept:  41.05693374499339 
Coefficients:  [-0.06343729  0.04916467 -0.          0.         -0.          0.9498107
  0.02090951 -0.66879     0.26420643 -0.01521159 -0.72296636  0.00824703
 -0.76111454]


In [54]:
modelL.score(x_test,y_test)

0.7118772394126163

In [55]:
lasso_scores = cross_val_score(modelL, x, y, cv=kf)
print(f'Lasso Regression Cross-Validation Scores: {lasso_scores}')
print(f'Mean Cross-Validation Score: {np.mean(lasso_scores)}')

Lasso Regression Cross-Validation Scores: [0.70577679 0.61681278 0.71865697 0.66089155 0.59844355]
Mean Cross-Validation Score: 0.660116328969042


Normally CV done on training dataset only. Here since we haven't splitted our data, it is done like this.

Different cv models for different situations

from sklearn.model_selection import StratifiedKFold
stratified_cv = StratifiedKFold(n_splits=5)
grid_search = GridSearchCV(model, param_grid, cv=stratified_cv, scoring='accuracy')

from sklearn.model_selection import LeaveOneOut
loo_cv = LeaveOneOut()
grid_search = GridSearchCV(model, param_grid, cv=loo_cv, scoring='accuracy')

from sklearn.model_selection import TimeSeriesSplit
time_series_cv = TimeSeriesSplit(n_splits=5)
grid_search = GridSearchCV(model, param_grid, cv=time_series_cv, scoring='accuracy')

from sklearn.model_selection import GroupKFold
group_cv = GroupKFold(n_splits=3)
grid_search = GridSearchCV(model, param_grid, cv=group_cv, scoring='accuracy', groups=group_labels)
